# Introducción

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Se necesita predecir si un cliente dejará el banco pronto. Se cuenta con los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Se creará un modelo con el máximo valor F1 posible (debe superar al menos 0.59)
Además, se medirá la métrica AUC-ROC y se la comparará con el valor F1.
  
 
 # Tabla de contenidos

* [1- Importación de librerias y carga de datos](#chapter1)

    
* [2 - Preparación de los datos](#chapter2)

    * [2 - 1 Preparación de datos para el modelo de regresion logística](#section_2_1)
    * [2 - 2 Preparación de datos para el modelo de arbol de desición y bosque aleatorio](#section_2_2) 
  
    
* [3- Entrenamiento y calculo de F1](#chapter3)

    * [3 - 1 Regresión Logística](#section_3_1)
    * [3 - 2 Arbol de desición](#section_3_2) 
    * [3 - 3 Bosque aleatorio](#section_3_3) 
  
  
* [4- Equilibrio de clases](#chapter4)

    * [4 - 1 Regresión Logística - con clases equilibradas](#section43_1)
    * [4 - 2 Arbol de desición- con clases equilibradas](#section_4_2) 
    * [4 - 3 Bosque aleatorio- con clases equilibradas](#section_4_3) 


* [5- Comprobación del modelo](#chapter5)


* [6- Evaluación de ajuste del umbral](#chapter6)


* [7- Conclusiones](#chapter7)

## Importación de librerias y análisis de datos <a class="anchor" id="chapter1"></a>

In [1]:
#importacion de librerias
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
data.columns = data.columns.str.lower()
data.head()

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
## Se rellenan valores usentes de Tenure. Se evaluan la media y la mediana
tenure_median = data['tenure'].median()
tenure_mean = data['tenure'].mean()
print("mediana", tenure_median)
print("media", tenure_mean)

mediana 5.0
media 4.997690023099769


In [7]:
# no hay valores atipicos en datos de TENURE, se rellenan condicionados a la variable BALANCE

tenure_mean = data.groupby(['age'])['tenure'].transform('median')

data['tenure'] = data['tenure'].fillna(tenure_mean)




In [8]:
# se verifica el tratamiento de ausentes
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [9]:
# Se verifica eliminación de ausentes
data['tenure'].value_counts().sum()

10000

In [10]:
#Se verifica presencia de datos duplicados
data.duplicated().sum()

0

## Preparación de los datos y creación de conjuntos de entrenamiento, validacion y test <a class="anchor" id="chapter2"></a>

- Se utilizarán modelos de clasificación dado que el objetivo es categeorico. 
- Para la transformacieon de las características categóricas, utilizaremos One Hot Encoder para el modelo de Regresión logística y Ordinal encoder para los modelos de Arbol de Decisión y bosque aleatorio
- En el caso del modelo de regresión logística, tambien se realizará un escalado de variables numéricas

### Preparación de datos para el modelo de regresión logística <a class="anchor" id="section_2_1"></a>

In [11]:
# Se aplica OHE a las columnas Geography y Gender
data_ohe = pd.get_dummies(data, drop_first = False, columns = ['geography', 'gender'])
data_ohe.head()

,rownumber,customerid,surname,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_France,geography_Germany,geography_Spain,gender_Female,gender_Male
0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,Onio,502,42,8.0,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,Boni,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,Mitchell,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [12]:
# - Se separan las caracteristicas del objetivo. A su vez, se elimina del analisis la columna 'Surname'

target_ohe = data_ohe['exited']
features_ohe = data_ohe.drop(['exited', 'surname'], axis=1)

In [13]:
# Se segmentan los datos en tres conjuntos (entrenamiento (60%), validacion (20%)y prueba (20%)). El tipo de plan es la variable objetivo


features_ohe_train, features_ohe_valid, target_ohe_train, target_ohe_valid = train_test_split(features_ohe, target_ohe, test_size = 0.4, random_state= 12345)
features_ohe_test, features_ohe_valid, target_ohe_test, target_ohe_valid  = train_test_split(features_ohe_valid, target_ohe_valid, test_size = 0.5, random_state= 12345)


In [14]:
# Se escalan las caracteristicas numericas (por ser un modelo basado en regresion)

numeric = ['creditscore','age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']
scaler = StandardScaler()
scaler.fit(features_ohe_train[numeric])
features_ohe_train[numeric] = scaler.transform(features_ohe_train[numeric])
features_ohe_valid[numeric] = scaler.transform(features_ohe_valid[numeric])
features_ohe_test[numeric] = scaler.transform(features_ohe_test[numeric])

/tmp/ipykernel_136/2433751039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_ohe_train[numeric] = scaler.transform(features_ohe_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


### Preparación de datos para el modelo de árbol de desición y bosque aleatorio  <a class="anchor" id="section_2_2"></a>

In [15]:
# Se utiiza  OrdinalEncoder para transformar variables categoricas

encoder =  OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns = data.columns)
data_ordinal

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,0.0,2736.0,1115.0,228.0,0.0,0.0,24.0,2.0,0.0,0.0,1.0,1.0,5068.0,1.0
1,1.0,3258.0,1177.0,217.0,2.0,0.0,23.0,1.0,743.0,0.0,0.0,1.0,5639.0,0.0
2,2.0,2104.0,2040.0,111.0,0.0,0.0,24.0,10.0,5793.0,2.0,1.0,0.0,5707.0,1.0
3,3.0,5435.0,289.0,308.0,0.0,0.0,21.0,1.0,0.0,1.0,0.0,0.0,4704.0,0.0
4,4.0,6899.0,1822.0,459.0,2.0,0.0,25.0,2.0,3696.0,0.0,1.0,1.0,3925.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995.0,1599.0,1999.0,380.0,0.0,1.0,21.0,6.0,0.0,1.0,1.0,0.0,4827.0,0.0
9996,9996.0,161.0,1336.0,125.0,0.0,1.0,17.0,12.0,124.0,0.0,1.0,1.0,5087.0,0.0
9997,9997.0,717.0,1570.0,318.0,0.0,0.0,18.0,9.0,0.0,0.0,0.0,1.0,2062.0,1.0
9998,9998.0,4656.0,2345.0,381.0,1.0,1.0,24.0,3.0,427.0,1.0,1.0,0.0,4639.0,1.0


In [ ]:
# Se segmentan los datos en tres conjuntos (entrenamiento (60%), validacion (20%)y prueba (20%)). El tipo de plan es la variable objetivo
features_ord = data_ordinal.drop('exited', axis=1)
target_ord = data_ordinal['exited']
features_ord_train, features_ord_valid, target_ord_train, target_ord_valid = train_test_split(features_ord, target_ord, test_size = 0.4, random_state= 12345)
features_ord_test, features_ord_valid, target_ord_test, target_ord_valid  = train_test_split(features_ord_valid, target_ord_valid, test_size = 0.5, random_state= 12345)


## Entrenamiento y calculo de F1_score  <a class="anchor" id="chapter3"></a>

### Modelo de Regresión logística  <a class="anchor" id="section_3_1"></a>

In [ ]:
#Entrenamiento y cálculo de F1 score para el modelo de regresión logística

model_log = LogisticRegression(random_state=54321, solver='liblinear')  # se inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model_log.fit(features_ohe_train, target_ohe_train) # se entrena el modelo en el conjunto de entrenamiento
predictions_log = model_log.predict(features_ohe_valid) # se predicen valores 
f1_score_log = f1_score(target_ohe_valid, predictions_log) # se calcula F1 score en el conjunto de validación

print("F1 score del modelo de regresión logística en el conjunto de validación:", f1_score_log)


### Modelo de árbol de desición  <a class="anchor" id="section_3_2"></a>

In [ ]:
#Entrenamiento y seleccion del mejor modelo de Arbol de decisión

best_model_decision_tree = None
best_result_decision_tree = 0
best_depth_decision_tree  = 0
for depth in range(1, 12):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth= depth) #se crea el modelo con la profundidad depth
    model_tree.fit(features_ord_train, target_ord_train) # se entrena el modelo 
    predictions_tree = model_tree.predict(features_ord_valid) # se obtienen las predicciones del modelo
    f1_score_tree = f1_score(target_ord_valid, predictions_tree) # se calcula F1 score
    if f1_score_tree > best_result_decision_tree:
        best_model_decision_tree = model_tree # guarda el modelo que corresponde a la mejor puntuacion de F1 score
        best_result_decision_tree = f1_score_tree # guarda la mejor puntuacion de F1 score
        best_depth_decision_tree = depth # guarda la profundidad que corresponde a la mejor puntuacion de F1 score

print("Mejor modelo 'Arbol de decision':")
print("F1_score:", best_result_decision_tree, "", "Profundidad:",  best_depth_decision_tree )



### Modelo de bosque aleatorio  <a class="anchor" id="section_3_3"></a>

In [ ]:
#Entrenamiento y seleccion del mejor modelo de bosque aleatorio

best_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 14): # selecciona el rango del hiperparámetro
    for depth in range(1,12):
        model_forest = RandomForestClassifier(random_state=54321, max_depth= depth, n_estimators=est) # configura el número de árboles
        model_forest.fit(features_ord_train, target_ord_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_forest = model_forest.predict(features_ord_valid)
        f1_score_forest = f1_score(target_ord_valid, predictions_forest) # calcula la puntuación de f1 en el conjunto de validación
        if f1_score_forest > best_score_random_forest:
            best_model_forest = model_forest
            best_score_random_forest = f1_score_forest# guarda la mejor puntuación de F1 en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de F1
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion den F1

print("Mejor modelo 'Bosque Aleatorio':")
print("F1_score:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)



- Los valores de F1 score no  superan el minimo de 0.59 solicitado en ninguno de los 3 modelos. Se analiza el equilibrio de clases para evaluar una mejora en los modelos

##  Equilibrio de clases <a class="anchor" id="chapter4"></a>

In [ ]:
#Evauluación del  equilibrio de clases
exited_frequency = data['exited'].value_counts(normalize=True)
exited_frequency

- Se observa un fuerte desequilibrio de clases, por lo que se procede a balancerla y entrenar los modelos con la clase balanceada

In [ ]:
# se crea una funcion para SOBREMUESTRO
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled



### Modelo regresión logística - con clases equilibradas  <a class="anchor" id="section_4_1"></a>

In [ ]:
# Se aplica la funcion de sobremuestreo al modelo de entrenamiento (creado para regresion logistica) y calcula el F1 score del modelo de regresion logistica con clase equilibrada
features_ohe_upsampled, target_ohe_upsampled = upsample(
    features_ohe_train, target_ohe_train, 10
)

model_log = LogisticRegression(random_state=12345, solver='liblinear')
model_log.fit(features_ohe_upsampled, target_ohe_upsampled)#entrenamiento del modelo
predicted_log = model_log.predict(features_ohe_valid)
F1_score_log =  f1_score(target_ohe_valid, predicted_log)
print('F1 score del modelo de regresión logística con clases equilibradas por sobremuestreo:',F1_score_log)


In [ ]:
# equilibrio de clases con hiperparametro classweight = balanced

model_log = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
model_log.fit(features_ohe_train, target_ohe_train) #entrenamiento del modelo
predicted_log = model_log.predict(features_ohe_valid)
F1_score_log = f1_score(target_ohe_valid, predicted_log)
print('F1 score del modelo de regresion logistica con clases equilibradas con classweight:', F1_score_log)


 - Se logro una mejora con el balanceo de clases en el modelo de regresion logistica pero aun está por debajo del mínimo requerido

### Modelo Arbol de desición - con clases equilibradas  <a class="anchor" id="section_4_2"></a>


In [ ]:
# Se aplica la funcion de sobremuestreo al modelo de entrenamiento (creado para arboles) y calcula el F1 score del modelo de arbol de desicion con clase equilibrada
features_ord_upsampled, target_ord_upsampled = upsample(
    features_ord_train, target_ord_train, 10)



In [ ]:
# Se busca el mejor modelo de árbol de desición con clases equilibradas por sobremuestro

best_model_decision_tree = None
best_result_decision_tree = 0
best_depth_decision_tree  = 0
for depth in range(1, 12):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth= depth) #se crea el modelo con la profundidad depth
    model_tree.fit(features_ord_upsampled, target_ord_upsampled) # se entrena el modelo 
    predictions_tree = model_tree.predict(features_ord_valid) # se obtienen las predicciones del modelo
    f1_score_tree = f1_score(target_ord_valid, predictions_tree) # se calcula F1
    if f1_score_tree > best_result_decision_tree:
        best_model_decision_tree = model_tree # guarda el modelo que corresponde a la mejor puntuacion de F1
        best_result_decision_tree = f1_score_tree # guarda la mejor puntuacion de accuracy
        best_depth_decision_tree = depth # guarda la profundidad que corresponde a la mejor puntuacion de F1

print("Mejor modelo 'Arbol de decision con clases equilibradas por sobremuestreo':")
print("F1_score:", best_result_decision_tree, "", "Profundidad:",  best_depth_decision_tree )




In [ ]:
# se busca el mejor modelo de árbol de desición con clases equilibradas con asignacion de class_weight

best_model_decision_tree = None
best_result_decision_tree = 0
best_depth_decision_tree  = 0
for depth in range(1, 12):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth= depth, class_weight = 'balanced') #se crea el modelo con la profundidad depth
    model_tree.fit(features_ord_train, target_ord_train) # se entrena el modelo 
    predictions_tree = model_tree.predict(features_ord_valid) # se obtienen las predicciones del modelo
    f1_score_tree = f1_score(target_ord_valid, predictions_tree) # se calcula la f1
    if f1_score_tree > best_result_decision_tree:
        best_model_decision_tree = model_tree # guarda el modelo que corresponde a la mejor puntuacion de f1
        best_result_decision_tree = f1_score_tree # guarda la mejor puntuacion de f1
        best_depth_decision_tree = depth # guarda la profundidad que corresponde a la mejor puntuacion de f1

print("Mejor modelo 'Arbol de decisión con clases equilibradas por sobremuestreo':")
print("F1_score:", best_result_decision_tree, "", "Profundidad:",  best_depth_decision_tree )

 - La mejora del modelo de arbol de decisión lograda con el equilibrio de clases es leve y aún no supera el mínimo de 0,59

### Modelo de bosque aleatorio - con clases equilibradas  <a class="anchor" id="section_4_3"></a>

In [ ]:
# se busca el mejor bosque aleatorio con clases equilibradas por sobremuestro

best_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 14): # selecciona el rango del hiperparámetro
    for depth in range(1,12):
        model_forest = RandomForestClassifier(random_state=54321, max_depth= depth, n_estimators=est) # configura el número de árboles
        model_forest.fit(features_ord_upsampled, target_ord_upsampled) # entrena el modelo en el conjunto de entrenamiento
        predictions_forest = model_forest.predict(features_ord_valid)
        f1_score_forest = f1_score(target_ord_valid, predictions_forest) # calcula la puntuación de f1 en el conjunto de validación
        if f1_score_forest > best_score_random_forest:
            best_score_random_forest = f1_score_forest# guarda la mejor puntuación de f1 en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de f1
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion den f1

print("Mejor modelo 'Bosque Aleatorio con clases equilibradas por sobremuestreo':")
print("F1_score:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)



In [ ]:
# se busca el mejor bosque aleatorio con clases equilibradas por ajuste de class_weight

best_score_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(1, 20): # selecciona el rango del hiperparámetro
    for depth in range(1,14):
        model_forest = RandomForestClassifier(random_state=54321, max_depth= depth, n_estimators=est, class_weight = 'balanced') # configura el número de árboles
        model_forest.fit(features_ord_train, target_ord_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_forest = model_forest.predict(features_ord_valid)
        f1_score_forest = f1_score(target_ord_valid, predictions_forest) # calcula la puntuación de f1 en el conjunto de validación
        if f1_score_forest > best_score_random_forest:
            best_score_random_forest = f1_score_forest# guarda la mejor puntuación de f1 en el conjunto de validación
            best_est_random_forest = est# guarda el número de estimadores que corresponden a la mejor puntuación de f1
            best_depth_random_forest = depth # guarda la la profundidad que corresponde a la mejor puntacion den f1

print("Mejor modelo 'Bosque Aleatorio con clases equilibradas por Classweight':")
print("F1_score:", best_score_random_forest, "", "Profundidad:",  best_depth_random_forest, "n_estimators:", best_est_random_forest)


###### - El modelo de bosque aleatorio es el que tuvo mayor mejora con el equilibrio de clase alcanzando un valore de 0.6, para una cantidad de 16 arboles y una profundidad de 8

## Comprobación de la calidad del modelo <a class="anchor" id="chapter5"></a>

In [ ]:
# calidad del modelo en el conjunto de prueba con los hiperparametros del mejor modelo de bosque aleatorio seleccionado

best_model_forest = RandomForestClassifier(random_state=54321, max_depth= 8, n_estimators= 16, class_weight = 'balanced') # configura el número de árboles
best_model_forest.fit(features_ord_train, target_ord_train) # entrena el modelo en el conjunto de entrenamiento
predictions_best_forest = best_model_forest.predict(features_ord_test)

f1_score_test = f1_score(target_ord_test, predictions_best_forest) 
        
print("F1_score del testo de validacion", f1_score_test)     
        

- EL f1 score de los datos de prueba da muy similar al del conjunto de validacion por lo que se comprueba la calidad del modelo seleccionado

In [ ]:
## se realiza una prueba de cordura: se compara el F1 score del modelo seleccionado con el de un modelo aleatorio

predictions_random = pd.Series(np.random.choice([0, 1], size=len(target_ord_test)), index=target_ord_test.index)

f1_score_random = f1_score(target_ord_test, predictions_random)
print("F1 score de prueba de cordura:", f1_score_random)

- EL f1 score obtenido con el modelo aleatorio es mucho menos en comparación con el obtenido por el modelo en  el del conjunto de validacion por lo que se comprueba la calidad del modelo seleccionado, habiendo pasado la prueba de cordura

In [ ]:
# calculo de roc - auc

probabilities_valid = best_model_forest.predict_proba(features_ord_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_ord_valid, probabilities_one_valid)

print(auc_roc)

- El valor de la auc - roc es muy elevado respecto del F1 score, podria deberse al desequilibrio de clases

## Evalución de ajuste de umbral  <a class="anchor" id="chapter6"></a>

In [ ]:
# Se evalua la posibilidad de ajustar el umbral dado que se trata de un objetivo de clases desequilibradas

probabilities_valid = best_model_forest.predict_proba(features_ord_valid)
probabilities_one_valid = probabilities_valid[:, 1]

   
for threshold in np.arange(0.4, 0.7, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1_score_forest = f1_score(target_ord_valid, predicted_valid)
    print(
        'Threshold = {:.2f} | F1_score = {:.3f}'.format(
            threshold, f1_score_forest))
    
    

In [ ]:

best_f1_score_forest = 0
for threshold in np.arange(0.4, 0.7, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1_score_forest = f1_score(target_ord_valid, predicted_valid)
    if f1_score_forest > best_f1_score_forest:
        best_f1_score_forest = f1_score_forest
        best_threhold_forest = threshold
        best_predicted = predicted_valid
print('threshold_best_f1:', best_threhold_forest, 'F1 score', best_f1_score_forest)


- EL mejor valor de F1 score se obtiene con umbral 0.5 (el msimo que por defecto), por lo que no se modifica el umbral para las predicciones

## Conclusiones <a class="anchor" id="chapter7"></a>

 - Los datos proporcionados contaban con valores ausentes en los datos de Ternure, los cuales fueron completados con la media de dichos valores
 - Dado que se trata de un objetivo de tipo catogorico, se utilizaron modelos de clasificacion para resolver el caso
 - Se transformaron las caracteristicas categoricas utilizando ordinal encoder para el caso de los modelos de arboles (arbol de desición y bosque aleatorio) y utilizando one hot encoder para el caso de modelo de regresion lineal
 - Las caracteristicas numericas se escalaron para el entrenamiento del modelo de regresion lineal (ya que no es necesario hacerlo para los modelos de arboles)
 - Se calcularon los valores de F1 score para los tres modelos, optimizando los hiperparametros de profundidad y cantidad de estimadores donde correspondia. En ninguno de los tres casos se supero el valor minimo requerido de F1 score de 0,58
 - Se procedio a equilibrar las clases del objetivo con dos metodos: ajuste del hiperparametro "class_weight" y mediante sobremuestero
 - Se calcularon para los tres modelos, utilizando ambos metodos de equilibrio de clases, los nuevos valores de F1 score:
     - En el modelo de regresion logistica , si bien se obtuvo una gran mejora del parametro , el valor aun quedo muy por debajo de minimo requerido
     - En el modelo de arbol de desicion, la mejora fue muy sutil, y no se alcanzo el minimo requerido
     - El valor de F1 alcanzado en el modelo de bosque aleatorio fue el unico que logro superar el minimo de f1 requerido, alcanzando un valor de 0.6, por lo cual , resulto ser el modelo seleccionado 
     
- Se ralizo una prueba de cordura, comparando el valor de F1 de un modelo aleatorio con el modelo seleccionado, siendo  el f1 score obtenido con el modelo aleatorio es mucho menor en comparación con el obtenido por el modelo en  el del conjunto de validacion por lo que se comprueba la calidad del modelo seleccionado, habiendo pasado la prueba de cordura
- Se realizo una comprobacion de la calidad del modelo, mediando el calculo de f1 en el conjunto de testeo, obteniendo un valor muy similar al obtenido ocn los datos de validacion lo que comprueba la calidad del modelo
- Se calculo el valor se auc  - roc, el cual resulto ser 0.84, un valor muy superior al del f1_score, por el desequilibrio de clases
- Se evaluo la convenciencia de ajustar el umbral, descartandose dicha posibilidad dado que el mayor valor de F1_score se da en el umbral por defecto del 0,5